# TPC-H Query 6

## Download Helper

In [1]:
import os
import boto3
import botocore
import numpy as np
import pandas as pd
import duckdb
bucket_name = 'alex-datasets'
prefix = 'tpch/sf-1/'

s3 = boto3.client('s3')

def download(bucket_name, key, fn):
  try:
    s3.download_file(bucket_name, key, fn)
    print(f'Downloaded {fn}')
  except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == '404':
      print(f'Object {key} does not exist!')
    else:
      raise

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Create directory if it doesn't exist
if not os.path.exists(prefix):
  os.makedirs(prefix)

# Download the files.
for obj in response.get('Contents', []):
  key = obj['Key']
  filename = os.path.join(prefix, key.split('/')[-1])

  if not os.path.exists(filename):
    download(bucket_name, key, filename)
  else:
    print(f'{filename} already exists.')

tpch/sf-1/customer.csv already exists.
tpch/sf-1/lineitem.csv already exists.
tpch/sf-1/nation.csv already exists.
tpch/sf-1/orders.csv already exists.
tpch/sf-1/part.csv already exists.
tpch/sf-1/partsupp.csv already exists.
tpch/sf-1/region.csv already exists.
tpch/sf-1/supplier.csv already exists.


## Query Implementation

```sql
select
  sum(l_extendedprice * l_discount) as revenue
from
  lineitem
where
  l_shipdate >= date '1994-01-01'
  and l_shipdate < date '1994-01-01' + interval '1' year
  and l_discount between 0.06 - 0.01 and 0.06 + 0.01
  and l_quantity < 24
```

In [3]:
# Load tables.
import pandas as pd
# Read the CSV file.
lineitem = pd.read_csv("./tpch/sf-1/lineitem.csv", names=["l_orderkey", "l_partkey", "l_suppkey", "l_linenumber", 
                                                          "l_quantity", "l_extendedprice", "l_discount", "l_tax",
                                                          "l_returnflag", "l_linestatus", "l_shipdate", 
                                                          "l_commitdate", "l_receiptdate", "l_shipinstruct", 
                                                          "l_shipmode", "l_comment"])

In [4]:
lineitem.head()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,155190,7706,1,17,21168.23,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the
1,1,67310,7311,2,36,45983.16,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold
2,1,63700,3701,3,8,13309.60,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,riously. regular@ express dep
3,1,2132,4633,4,28,28955.64,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de
4,1,24027,1534,5,24,22824.48,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re


In [4]:
# Convert columns to datetime format
import time

date_columns = ["l_shipdate", "l_commitdate", "l_receiptdate"]
print(len(lineitem))
def tpch_query_6(df):
  df[date_columns] = df[date_columns].apply(pd.to_datetime)
  date_str1 = '1994-01-01'
  date_str2 = '1995-01-01'
  date_time1 = pd.to_datetime(date_str1)
  date_time2 = pd.to_datetime(date_str2)
  df = df[(df["l_shipdate"] < date_time2) & (df["l_shipdate"] >= date_time1)]
  df = df[(df["l_discount"] < 0.07 + 0.000001) & (df["l_discount"] > 0.05 - 0.000001) & (df["l_quantity"] < 24)]
  df["sum_col"] = df["l_extendedprice"] * df["l_discount"]
  return sum(df["sum_col"])

st = time.perf_counter()
query_result = tpch_query_6(lineitem)
et = time.perf_counter()
print("time taken", et - st)
print(query_result)

6001215
time taken 3.333437404999131
123141078.22829895


## Checker

In [5]:
# This part checks whether your output is correct.
import numpy as np
assert np.isclose(query_result, 123141078.2283, atol=1e-6)

NameError: name 'query_result' is not defined

### Create the lineitem table

In [6]:
lineitem_table = """
CREATE TABLE IF NOT EXISTS lineitem (
    l_orderkey INTEGER,
    l_partkey INTEGER,
    l_suppkey INTEGER,
    l_linenumber INTEGER,
    l_quantity DECIMAL,
    l_extendedprice DECIMAL,
    l_discount DECIMAL,
    l_tax DECIMAL,
    l_returnflag CHAR(1),
    l_linestatus CHAR(1),
    l_shipdate DATE,
    l_commitdate DATE,
    l_receiptdate DATE,
    l_shipinstruct VARCHAR,
    l_shipmode VARCHAR,
    l_comment VARCHAR
) 
"""
duckdb.query(lineitem_table)
insert_query_lineitem = """INSERT INTO lineitem SELECT * FROM read_csv("tpch/sf-1/lineitem.csv")"""
duckdb.query(insert_query_lineitem)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
import time
st_time = time.time()
query6 = """
        select
          sum(l_extendedprice * l_discount) as revenue
        from
          lineitem
        where
          l_shipdate >= date '1994-01-01'
          and l_shipdate < date '1994-01-01' + interval '1' year
          and l_discount between 0.06 - 0.01 and 0.06 + 0.01
          and l_quantity < 24
        """
duckdb.query(query6)
print(f"time taken: {time.time() - st_time}")

CatalogException: Catalog Error: Table with name lineitem does not exist!
Did you mean "pg_index"?

In [12]:
### Save sorted csv

In [14]:
sort_query = """COPY (SELECT * FROM lineitem ORDER BY l_shipdate) TO 'output.csv' (HEADER, DELIMITER ',')"""
duckdb.query(sort_query)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### PERFORMING Q6 using orc files

In [6]:
import pandas as pd
import duckdb
import math
result4 = pd.read_orc("/media/suman/Transcend/UTN/UTN_HDD/AIR/semester2/cloud_computing/week2/orc/warehouse/default/lineitem/data/00003-16-2b00eabe-fd67-4e9a-b5b3-0be7eaf88f09-0-00001.orc")  
result5 = pd.read_orc("/media/suman/Transcend/UTN/UTN_HDD/AIR/semester2/cloud_computing/week2/orc/warehouse/default/lineitem/data/00004-17-2b00eabe-fd67-4e9a-b5b3-0be7eaf88f09-0-00001.orc")  
result6 = pd.read_orc("/media/suman/Transcend/UTN/UTN_HDD/AIR/semester2/cloud_computing/week2/orc/warehouse/default/lineitem/data/00005-18-2b00eabe-fd67-4e9a-b5b3-0be7eaf88f09-0-00001.orc")  

In [11]:
query6 = """
        select
          sum(l_extendedprice * l_discount) as revenue
        from
          lineitem
        where
          l_shipdate >= date '1994-01-01'
          and l_shipdate < date '1994-01-01' + interval '1' year
          and l_discount between 0.06 - 0.01 and 0.06 + 0.01
          and l_quantity < 24
        """
lineitem_table = """
CREATE TABLE IF NOT EXISTS lineitem AS SELECT * FROM result6
"""
duckdb.query(lineitem_table)
result = duckdb.query(query6)

In [8]:
math.isclose(68462078.48259994 + 46402803.57049994 + 8276196.175199989, 123141078.22829895)

True

In [24]:
result.fetchone()[0]

8276196.175199989